In [1]:

import os
if not os.path.exists('hw2-dl-audio'):
    !git clone https://github.com/Vladislav747/hw2-dl-audio.git
    %cd hw2-dl-audio
else:
    %cd hw2-dl-audio


Cloning into 'hw2-dl-audio'...
remote: Enumerating objects: 798, done.
remote: Counting objects: 100% (798/798), done.
remote: Compressing objects: 100% (443/443), done.
remote: Total 798 (delta 455), reused 656 (delta 313), pack-reused 0 (from 0)
Receiving objects: 100% (798/798), 1.51 MiB | 10.78 MiB/s, done.
Resolving deltas: 100% (455/455), done.
/content/hw2-dl-audio


In [15]:

!pip install -q torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0
!pip install -q soundfile numpy==1.26.4 scipy torch_audiomentations tqdm matplotlib pandas
!pip install -q hydra-core wget datasets editdistance pyctcdecode
!pip install -q comet_ml wandb


## 2. Скачивание предобученных весов


In [11]:
!pip install -q gdown

import os
import gdown
import zipfile

url = f"https://drive.google.com/uc?id=11enEtJOL1zx2CveLQg7MoOqAAHWs3L1o"

os.makedirs("saved", exist_ok=True)

archive_path = "saved/weights_archive.zip"
print("Скачивание весов")
gdown.download(url, archive_path, quiet=False)

print("Распаковка архива...")
if archive_path.endswith('.zip'):
    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        zip_ref.extractall("saved")

os.remove(archive_path)
print("Веса распакованы в папку saved")


Скачивание весов


Downloading...
From (original): https://drive.google.com/uc?id=11enEtJOL1zx2CveLQg7MoOqAAHWs3L1o
From (redirected): https://drive.google.com/uc?id=11enEtJOL1zx2CveLQg7MoOqAAHWs3L1o&confirm=t&uuid=1ee7a17b-f5b7-44a3-954b-1bd7a795edb3
To: /content/hw2-dl-audio/saved/weights_archive.zip

  0%|          | 0.00/1.35G [00:00<?, ?B/s]
  0%|          | 4.72M/1.35G [00:00<00:29, 45.5MB/s]
  1%|▏         | 17.3M/1.35G [00:00<00:14, 91.9MB/s]
  2%|▏         | 26.7M/1.35G [00:00<00:16, 82.4MB/s]
  3%|▎         | 35.1M/1.35G [00:00<00:18, 71.2MB/s]
  4%|▍         | 53.0M/1.35G [00:00<00:13, 95.0MB/s]
  6%|▌         | 75.5M/1.35G [00:00<00:09, 132MB/s] 
  7%|▋         | 89.7M/1.35G [00:00<00:13, 92.0MB/s]
  8%|▊         | 107M/1.35G [00:01<00:12, 102MB/s]  
  9%|▉         | 120M/1.35G [00:01<00:12, 99.9MB/s]
 10%|▉         | 131M/1.35G [00:01<00:15, 78.8MB/s]
 11%|█         | 149M/1.35G [00:01<00:13, 85.7MB/s]
 12%|█▏        | 159M/1.35G [00:01<00:13, 85.3MB/s]
 13%|█▎        | 170M/1.35G [00:01<00:

Распаковка архива...
Веса распакованы в папку saved


## 3. Быстрый тест на onebatchtest датасете


In [ ]:
# Запуск обучения на маленьком датасете для проверки
!python3 train.py -cn=deepspeech2 \
  datasets=onebatchtest \
  trainer.n_epochs=1 \
  trainer.override=True \
  writer=cometml


## 4. Обучение на полном датасете (train-clean-360)

Показываю как запускал - это версия без augs

In [ ]:
# Раскомментируйте для запуска полного обучения
# python3 train.py -cn=deepspeech2_360 \
#     datasets=train_clean_360 \
#     trainer.n_epochs=200 \
#     trainer.override=True \
#     writer=cometml \
#     writer.run_name="deepspeech2_360" \
#     2>&1


Версия с augs - сильная вероятность использования

In [ ]:

# python3 train.py -cn=deepspeech2_augs_strong_360 \
#     datasets=train_clean_360 \
#     trainer.n_epochs=200 \
#     trainer.override=True \
#     writer=cometml \
#     writer.run_name="deepspeech2_augs_strong_360" \
#     2>&1

## 5. Запуск Inference


In [24]:
# Запуск inference на обученной модели
# Убедитесь, что у вас есть сохраненная модель в saved/model_best.pth

#### Оценка модели на test-clean(без augs)
!python3 inference.py \
    -cn=inference \
    datasets=test_clean \
    inferencer.from_pretrained="saved/saved/deepspeech2_360/model_best.pth" \
    inferencer.save_path="predictions_test_clean"



kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
LM path is not provided
DeepSpeech2(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 2), padding=(10, 5))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 96, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
    (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (rnns): Sequential(
    (0): NormGRU(
      (gru): GRU(1536, 512, dropout=0.1, bidirectional=True)
  

In [ ]:
#### Оценка модели на test-other(с augs)

!python3 inference.py \
    -cn=inference \
    datasets=test_other \
    inferencer.from_pretrained="saved/saved/deepspeech2_augs_strong_360/model_best.pth" \
    inferencer.save_path="predictions_test_other"


kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
LM path is not provided
DeepSpeech2(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 2), padding=(10, 5))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 96, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
    (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (rnns): Sequential(
    (0): NormGRU(
      (gru): GRU(1536, 512, dropout=0.1, bidirectional=True)
  